In [1]:
from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances

In [2]:
def k_centers_selection(df, k, random_state=42):
    """
    Selecciona k muestras del conjunto de datos utilizando el algoritmo de K-Centers.
    
    Parameters:
    df (pandas DataFrame): Conjunto de datos de entrada sin la columna de etiquetas.
    k (int): Número de muestras a seleccionar.
    random_state (int): Semilla para el generador de números aleatorios.
    
    Returns:
    numpy array: Índices de las muestras seleccionadas.
    """
    np.random.seed(random_state)  # Fijar la semilla aleatoria para obtener resultados reproducibles
    data = df.values  # Convertir el DataFrame a un array numpy
    n_samples = data.shape[0]
    distances = pairwise_distances(data)
    
    # Inicializar el primer centro aleatoriamente
    first_center = np.random.choice(n_samples)
    centers = [first_center]
    
    for _ in range(1, k):
        # Calcular la distancia mínima de cada punto a los centros actuales
        min_distances = np.min(distances[:, centers], axis=1)
        # Seleccionar el punto más lejano de los centros actuales
        new_center = np.argmax(min_distances)
        centers.append(new_center)
    
    return centers

def select_representative_samples(df, label_column, k_per_class, columnas_sin_null, random_state=42):
    """
    Selecciona muestras representativas de cada clase utilizando K-Centers.
    
    Parameters:
    df (pandas DataFrame): Conjunto de datos de entrada con etiquetas.
    label_column (str): Nombre de la columna de etiquetas.
    k_per_class (int): Número de muestras a seleccionar por clase.
    columnas_sin_null (list): Lista de columnas a utilizar para la selección.
    random_state (int): Semilla para el generador de números aleatorios.
    
    Returns:
    pandas DataFrame: DataFrame con las muestras seleccionadas.
    """
    selected_indices = []
    unique_labels = df[label_column].unique()
    
    for label in unique_labels:
        subset = df[df[label_column] == label].drop(columns=[label_column])
        selected = k_centers_selection(subset[columnas_sin_null].iloc[:, 3:], k_per_class, random_state)  # No van columnas id
        selected_indices.extend(df[df[label_column] == label].iloc[selected].index)
    
    return df.loc[selected_indices]

In [3]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
cur = conn.cursor()

In [4]:
segmentos_modos = []
meses = ['sept2023', 'abr2024', 'mayo2024']
meses = ['abr2024']
meses = ['sept2023', 'abr2024'] # Bicicleta nuevos
meses = ['sept2023']
for mes in meses:
    tablas_modos = [f'reglas_segmentos_caminata_{mes}', f'reglas_segmentos_bicicleta_{mes}', f'reglas_segmentos_bus_{mes}', f'reglas_segmentos_automovil_{mes}']

    
    for tabla_modos in tablas_modos:

        if mes == 'sept2023' or (mes != 'abr2024' and ('automovil' in tabla_modos) or (mes != 'sept2023' and ('bicicleta' in tabla_modos))):
            consulta_sql = f'''
                            SELECT o.*
                                FROM estadisticos_segmentos_{mes} o
                                JOIN {tabla_modos} i
                                ON o.tripid = i.tripid
                                AND o.segid = i.segid
                                AND o.uid = i.uid;
                            '''
        
            cur.execute(consulta_sql)
            df_modo = pd.DataFrame(cur.fetchall().copy(), columns=[desc[0] for desc in cur.description])
            df_modo['etiqueta'] = tabla_modos.split("_")[2].upper()

            segmentos_modos.append(df_modo)

# Concatenar los DataFrames de la lista en uno solo, ignorando los índices originales
df_modos = pd.concat(segmentos_modos, ignore_index=True)

num_elementos_por_grupo = df_modos.groupby('etiqueta').size()
print(num_elementos_por_grupo)

etiqueta
AUTOMOVIL      680
BICICLETA      110
BUS            382
CAMINATA     15117
dtype: int64


In [5]:
df_modos = df_modos.dropna()
num_elementos_por_grupo = df_modos.groupby('etiqueta').size()
print(num_elementos_por_grupo)

etiqueta
AUTOMOVIL      567
BICICLETA      110
BUS            382
CAMINATA     14820
dtype: int64


In [6]:
# ----Verificar qué columnas tienen valores nulos
columnas_con_null = df_modos.columns[df_modos.isnull().any()]
# Tomar solo columnas sin nulos
columnas_sin_null = [col for col in df_modos.columns if col not in columnas_con_null]

# ---- Submuestrear
k_per_class = num_elementos_por_grupo.min()  # Número de muestras a seleccionar por clase
#k_per_class = 140  # Número de muestras a seleccionar por clase
selected_samples = select_representative_samples(df_modos, 'etiqueta', k_per_class, columnas_sin_null[:-1], random_state=42)
num_elementos_por_grupo = selected_samples.groupby('etiqueta').size()
print(num_elementos_por_grupo)

etiqueta
AUTOMOVIL    110
BICICLETA    110
BUS          110
CAMINATA     110
dtype: int64


In [7]:
# ---- Agregar columnas para caracteristicas nuevas
consulta_sql = f'''DROP TABLE IF EXISTS datos_uda_reglas;
                  CREATE TABLE datos_uda_reglas (
                      tripid INTEGER,
                      segid INTEGER,
                      uid VARCHAR(255),
                      p25_speed FLOAT, 
                      p75_speed FLOAT,
                      max_speed FLOAT,
                      min_speed FLOAT,
                      mean_speed FLOAT,
                      median_speed FLOAT,
                      var_speed FLOAT,
                      p25_aceleration FLOAT, 
                      p75_aceleration FLOAT,
                      max_aceleration FLOAT,
                      min_aceleration FLOAT,
                      mean_aceleration FLOAT,
                      median_aceleration FLOAT,
                      var_aceleration FLOAT,
                      p25_bearing FLOAT, 
                      p75_bearing FLOAT,
                      max_bearing FLOAT,
                      min_bearing FLOAT,
                      mean_bearing FLOAT,
                      median_bearing FLOAT,
                      var_bearing FLOAT,
                      p25_time_stop FLOAT, 
                      p75_time_stop FLOAT,
                      max_time_stop FLOAT,
                      min_time_stop FLOAT,
                      mean_time_stop FLOAT,
                      median_time_stop FLOAT,
                      var_time_stop FLOAT,
                      stops INTEGER,
                      etiqueta VARCHAR(50)
                  );
                '''
cur.execute(consulta_sql)
conn.commit()

# ---- Guardar estadisticos de distribucion datos muestreados
for idx, row in selected_samples.iterrows():
    consulta_sql = """
    INSERT INTO {} VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s
    )
    """.format(f'datos_uda_reglas') # Una sola tabla para todos los samples

    cur.execute(consulta_sql, row)
    conn.commit()

conn.close()

C:\Users\Mateo\AppData\Local\Temp\ipykernel_20580\1001500720.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cur.execute(consulta_sql, row)
